In [ ]:
#This program can be run offline but requires the proper input text file: https://www.ngdc.noaa.gov/stp/satellite/goes/doc/SPE.txt
import requests #For html access
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup #For html parsing

In [ ]:
def read_EPS_data(file_name='GOES_SEP_raw.txt'):
    '''
    This function reads the raw data file for the SEP events recorded by GOES and stores relevant information to a
    separate file, in a much more easily readable format. This file must be in the same directory as this script.
    The data can be found at:
    https://www.ngdc.noaa.gov/stp/satellite/goes/doc/SPE.txt
    This file is difficult to parse since it does not have a single standard format. As such, there are many if-
    statements in this function that are not necessary on better-formatted data. Even with these checks, the
    formatting of the input file is so inconsistent that some information is still not read properly. The output
    file of this function will therefore contain only 225 events, whereas the input file contains about 266 events.
    
    Arguments
    file_name:
    The name of the .txt file containing the data from GOES. 'GOES_SEP_raw.txt' by default.
    output_file_name: The name of the .txt file to which this function will write its output dataframe.
    'GOES_SEP_flares.txt' by default.
    
    Returns
    Does not return anything. Instead, writes to a file in the same directory as this script.
    '''
    
    #Read in all the data from the input file as a list of lines (each line is one event)
    with open(file_name, 'r') as file:
        text = file.read()
    contents = text.splitlines()
    
    all_events = pd.DataFrame([]) #Initialize a dataframe in which to store the output data
    col_num = 0  #Initialize an int with which to name columns of all_events. Ensures no two columns have the same name.
    for line in contents:
        line = line.replace('\t','') #Remove tab characters
        if len(line.replace(' ','')) == 4 and line != 'none': #If the line contains just a year (e.g. 1995)
            year = line.replace(' ','') #Remove spaces
        else:
            if len(line) > 5:
                s=0
                s2=0
                #s and s2 are variables that help account for shifts in the location of certain bytes in the strings of certain events
                if int(year) == 2002 or int(year) == 2003:
                    s=9
                    s2=16
                if int(year) >= 2004:
                    s2=1
                peak_month = line[56-s:59-s]
                peak_day = line[60-s:62-s]
                peak_time = line[63-s:67-s]
                flare_loc = line[82-s2:88-s2]
                flare_class = line[70-s:72-s]
                this_event = [year, peak_month, peak_day, peak_time, flare_loc, flare_class]

                months_dict = {1:"Jan",2:"Feb",3:"Mar",4:"Apr",5:"May",6:"Jun",7:"Jul",8:"Aug",9:"Sep",10:"Oct",11:"Nov",12:"Dec"}

                if peak_month in months_dict.values():  #Ensures that rows with hard-to-read months don't make it into the dataframe
                    all_events = pd.concat([all_events,pd.DataFrame(this_event,columns=[col_num])], axis=1) #Add data for the considered event to the final dataframe
                    col_num+=1

    all_events = all_events.rename(index={0: "year", 1: "month", 2: "day", 3: "peak_time", 4: "location", 5: "class"}) #Rename the rows of the dataframe to be more descriptive

    all_events.to_csv(output_file_name) #Write the dataframe to file

In [ ]:
read_EPS_data()